In [ ]:
from itertools import cycle
from typing import Optional, Union

import os
from datetime import datetime, timedelta
from time import sleep
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from glob import glob
import requests_cache
import numpy as np

In [ ]:
data_dir = '../database/'
# df = pd.read_parquet(data_dir + 'latest.parquet')
df_smard = pd.read_parquet(data_dir + 'smard/' + 'history.parquet')
df_om = pd.read_parquet(data_dir + 'openmeteo/' + 'history.parquet')
df_om_f = pd.read_parquet(data_dir + 'openmeteo/' + 'forecast.parquet')
df_es = pd.read_parquet(data_dir + 'epexspot/' + 'history.parquet')

In [ ]:
df_smard.tail(n=5)

In [ ]:
def plot_gen_load_with_forecasts(df_smard_):
    
    df_smard_ = df_smard_[~(df_smard_[['total_grid_load','residual_load','solar','wind_offshore','wind_onshore']] == 0).all(axis=1)]
    
    fig,axes = plt.subplots(figsize=(10,10), nrows=6, sharex='all')
    ax = axes[0]
    
    ax.plot(df_smard_.index,df_smard_['total_grid_load'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['total_grid_load_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Total Load')
    
    ax = axes[1]
    ax.plot(df_smard_.index,df_smard_['residual_load'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['residual_load_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Residual Load')
    
    ax = axes[2]
    ax.plot(df_smard_.index,df_smard_['solar'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['solar_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Solar Generation')
    
    ax = axes[3]
    ax.plot(df_smard_.index,df_smard_['wind_offshore'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['wind_offshore_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Wind Off-shore')
    
    ax = axes[4]
    ax.plot(df_smard_.index,df_smard_['wind_onshore'], color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['wind_onshore_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Wind On-shore')
    
    ax = axes[5]
    ax.plot(df_smard_.index,df_smard_[['biomass','hydropower','lignite','hard_coal','natural_gas','pumped_storage','other_conventional','other_renewables']].aggregate(func='sum',axis=1), color='blue', label='Actual')
    ax.plot(df_smard_.index,df_smard_['other_gen_forecasted'], color='gray', label='Forecast')
    ax.set_ylabel('Other Generation')
    
    plt.tight_layout()
    plt.show()
plot_gen_load_with_forecasts(df_smard.tail(30*24))

In [ ]:
def visualize_energy_data(df):
    # Ensure datetime index
    df_month = df.tail(30*24)

    # List of countries
    countries = ['austria', 'belgium', 'czechia', 'denmark', 'france', 'luxembourg',
                 'netherlands', 'norway', 'poland', 'sweden', 'switzerland']

    # Sum '_export' and '_import' columns for each country
    df_flows = pd.DataFrame()
    for country in countries:
        df_flows[country] = df_month[f'{country}_export'] + df_month[f'{country}_import']

    # Remaining columns
    remaining_columns = ['other_gen', 'residual_load_forecast', 'solar', 'total_gen',
                         'total_grid_load', 'wind_offshore', 'wind_onshore']
    df_remaining = df_month[remaining_columns]


    # Plotting
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

    # First panel: stacked bar chart of df_flows
    df_flows.plot(kind='bar', stacked=True, ax=axes[0], width=1.0, linewidth=0)

    axes[0].set_title('Hourly Energy Flows by Country')
    axes[0].set_ylabel('Total Energy Flow')
    axes[0].legend(loc='upper left', bbox_to_anchor=(1.02, 1))
    axes[0].set_xlabel('')

    # Adjust x-axis ticks on the bar chart
    num_ticks = len(df_flows)
    tick_positions = range(0, num_ticks, 24)  # Show a tick every 24 bars (once per day)
    tick_labels = df_flows.index[::24].strftime('%Y-%m-%d')
    axes[0].set_xticks(tick_positions)
    axes[0].set_xticklabels(tick_labels, rotation=45, ha='right')

    # Second panel: lines of remaining columns
    df_remaining.plot(ax=axes[1])
    axes[1].set_title('Hourly Other Energy Data')
    axes[1].set_ylabel('Value')
    axes[1].legend(loc='upper left', bbox_to_anchor=(1.02, 1))

    # Adjust x-axis ticks on the second panel
    axes[1].xaxis.set_major_locator(mdates.DayLocator(interval=3))
    axes[1].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')
    axes[1].set_xlabel('Date')

    plt.tight_layout()
    plt.show()
visualize_energy_data(df_smard)

E

In [ ]:
df_es.tail(24*30).plot(figsize=(16,5))

In [ ]:
df_es.tail()

In [ ]:
def visualize_time_series(df, df_f, suffix, figsize=(15, 20)):
    """
    Visualizes each column with a specified suffix in separate panels with a shared X-axis.

    Parameters:
    - df (pd.DataFrame): DataFrame with time-series data indexed by pd.Timestamp.
    - suffix (str): Suffix to filter the columns of interest.
    - figsize (tuple): Size of the figure for the plots.
    """
    # Filter columns based on the suffix
    filtered_columns = [col for col in df.columns if col.endswith(suffix)]

    if not filtered_columns:
        raise ValueError(f"No columns found with suffix '{suffix}' in the DataFrame.")

    num_columns = len(filtered_columns)

    # Create a shared X-axis for all subplots
    fig, axes = plt.subplots(num_columns, 1, figsize=figsize, sharex=True)
    if num_columns == 1:
        axes = [axes]  # Ensure axes is always iterable

    for ax, column in zip(axes, filtered_columns):
        ax.plot(df.index, df[column], label=column)
        ax.plot(df_f.index, df_f[column], ls='--')
        ax.set_ylabel(column.replace(suffix, '').replace('_', ' ').capitalize())
        ax.legend(loc='upper right')
        ax.grid(True)

    # Set shared X-axis label
    plt.xlabel("Time")
    plt.tight_layout()
    plt.show()
visualize_time_series(df=df_om,df_f = df_om_f, suffix='hsee')

def

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_multivariate_timeseries(df, df_fore):
    # Create a subplot with 3 rows
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)

    # Plot 'DA_auction_price' on the first subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['DA_auction_price'], name='DA Auction Price'),
                  row=1, col=1)

    # Plot 'total_gen', 'total_grid_load', 'residual_load_forecast' on the second subplot
    fig.add_trace(go.Scatter(x=df.index, y=df['total_gen'], name='Total Generation'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['total_grid_load'], name='Total Grid Load'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['residual_load_forecast'], name='Residual Load Forecast'), row=2, col=1)

    # Plot 'temperature_2m_hsee' and 'temperature_2m_solw' on the third subplot
    for df_,ls in zip([df, df_fore],['solid', 'dash']):
        fig.add_trace(go.Scatter(
            x=df_.index,
            y=df_['temperature_2m_hsee'],
            name='Temperature 2m HSEE',
            line=dict(dash=ls,color='blue')
        ), row=3, col=1)

        fig.add_trace(go.Scatter(
            x=df_.index,
            y=df_['temperature_2m_solw'],
            name='Temperature 2m SOLW',
            line=dict(dash=ls,color='green')
        ), row=3, col=1)
    # Add a vertical line for today across all subplots
    # today_line = dict(type='line', x0=pd.Timestamp.today(), y0=0, x1=pd.Timestamp.today(),
    #                   y1=1, xref='x', yref='paper', line=dict(color='black', width=2, dash='dash'))
    # fig.add_shape(today_line, row='all', col=1)
    fig.add_vline(x=pd.Timestamp.today(), line_width=1, line_dash="dash", line_color="gray")

    # Update layout
    fig.update_layout(height=900, title='Multivariate Time Series Analysis',
                      xaxis_title='DateTime', yaxis_title='Values')

    # Show the figure
    fig.show()
plot_multivariate_timeseries(df=df_hist.tail(1000), df_fore = df_fore)

# Plot SMARD data

In [ ]:
from data_modules.collect_data_smard import DataEnergySMARD
countries = list(DataEnergySMARD.country_map.keys())
# Create a subplot with 3 rows
fig = make_subplots(rows=len(countries), cols=1, shared_xaxes=True, vertical_spacing=0.02)

df_hist_ = df_hist.tail(4*7*24)
for i, country in enumerate(countries):
    fig.add_trace(go.Scatter(
        x=df_hist_.index,
        y=df_hist_[f'{country}_export'],
        name=f'{country}',
        line=dict(color='blue')  # Set the line color here
    ), row=i+1, col=1)
    fig.add_trace(go.Scatter(
        x=df_hist_.index,
        y=df_hist_[f'{country}_import'],
        name=None,#f'{country}',
        line=dict(color='red')  # Set the line color here
    ), row=i+1, col=1)
# Update layout
fig.update_layout(height=1800, title='International trade',
xaxis_title='DateTime', yaxis_title='Values')

# Show the figure
fig.show()